<a href="https://colab.research.google.com/github/DonutUE/Barbershop_Project/blob/main/Barbershop_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import time
import shutil
import sys
import threading
from google.colab import drive

# 1. Google Drive
try:
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/barbershop_clean.sqlite3'
except:
    drive_path = None

# 2. Очищення
try: os.chdir('/content')
except: pass
!pkill -9 ngrok
if os.path.exists("/root/.config/ngrok"): shutil.rmtree("/root/.config/ngrok")

# 3. Бібліотеки
try:
    import django
    from pyngrok import ngrok, conf
except ImportError:
    !pip install -q django pyngrok
    import django
    from pyngrok import ngrok, conf

from django.core.management import call_command
from django.db.models import Avg

# 4. Проект
project_name = 'barbershop_project'
project_path = os.path.join('/content', project_name)
if os.path.exists(project_path): shutil.rmtree(project_path)

# 5. Ngrok
token = "37oF8e3aJm0DsXuUVZMoqRietgs_4XyvCMiBvgaN8AreBgjQj"
pyngrok_config = conf.PyngrokConfig(auth_token=token, region="us")
conf.set_default(pyngrok_config)

try:
    public_url = ngrok.connect(8000).public_url
except Exception as e:
    sys.exit()

!django-admin startproject $project_name
os.chdir(project_path)
inner_path = os.path.join(project_path, project_name)

# 6. Settings (CSRF fix)
settings_file = os.path.join(inner_path, 'settings.py')
with open(settings_file, 'r') as f: content = f.read()
content = content.replace("INSTALLED_APPS = [", "INSTALLED_APPS = ['barbershop_project', ")
content = content.replace("ALLOWED_HOSTS = []", "ALLOWED_HOSTS = ['*']")
content += f"\nCSRF_TRUSTED_ORIGINS = ['{public_url}', 'https://*.ngrok-free.dev', 'https://*.ngrok-free.app']\n"
content += "SECURE_PROXY_SSL_HEADER = ('HTTP_X_FORWARDED_PROTO', 'https')\n"
content += "\nLOGIN_URL = '/login/'\n"
with open(settings_file, 'w') as f: f.write(content)

# 7. Models
models_code = """
from django.db import models
from django.contrib.auth.models import User

class Location(models.Model):
    city = models.CharField(max_length=100); address = models.CharField(max_length=200)
    def __str__(self): return f"{self.city}, {self.address}"

class Master(models.Model):
    name = models.CharField(max_length=100)
    specialty = models.CharField(max_length=100)
    location = models.ForeignKey(Location, on_delete=models.CASCADE)
    photo_url = models.CharField(max_length=500, default="https://via.placeholder.com/300")
    rating = models.FloatField(default=5.0)
    reviews_count = models.IntegerField(default=0)
    bio = models.TextField(default="Професіонал своєї справи.")
    def __str__(self): return self.name

class Service(models.Model):
    name = models.CharField(max_length=100); price = models.IntegerField(); description = models.TextField(blank=True)
    def __str__(self): return f"{self.name} - {self.price}"

class Booking(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    service = models.ForeignKey(Service, on_delete=models.CASCADE)
    master = models.ForeignKey(Master, on_delete=models.CASCADE)
    location = models.ForeignKey(Location, on_delete=models.CASCADE)
    phone = models.CharField(max_length=20)
    wishes = models.TextField(blank=True, null=True)
    date_time = models.DateTimeField()
    created_at = models.DateTimeField(auto_now_add=True)

class Review(models.Model):
    master = models.ForeignKey(Master, on_delete=models.CASCADE, related_name='reviews')
    user = models.ForeignKey(User, on_delete=models.CASCADE)
    rating = models.IntegerField(choices=[(i, i) for i in range(1, 6)])
    text = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
"""
with open(os.path.join(inner_path, 'models.py'), 'w') as f: f.write(models_code)

# 8. Admin
admin_code = """
from django.contrib import admin
from .models import Service, Booking, Location, Master, Review
@admin.register(Service)
class ServiceAdmin(admin.ModelAdmin): list_display = ('name', 'price')
@admin.register(Master)
class MasterAdmin(admin.ModelAdmin): list_display = ('name', 'rating')
@admin.register(Booking)
class BookingAdmin(admin.ModelAdmin): list_display = ('user', 'phone', 'master', 'date_time')
@admin.register(Review)
class ReviewAdmin(admin.ModelAdmin): list_display = ('master', 'user', 'rating')
"""
with open(os.path.join(inner_path, 'admin.py'), 'w') as f: f.write(admin_code)

# 9. Views
views_code = """
from django.http import HttpResponse
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib.auth import login, logout
from django.contrib.auth.forms import UserCreationForm, AuthenticationForm
from django.contrib.auth.decorators import login_required
from django.views.decorators.csrf import csrf_exempt
from django.db.models import Avg
from .models import Service, Booking, Master, Location, Review

def home(request):
    sort = request.GET.get('sort', '')
    if sort == 'asc': services = Service.objects.all().order_by('price')
    elif sort == 'desc': services = Service.objects.all().order_by('-price')
    else: services = Service.objects.all()

    masters = Master.objects.all()
    locations = Location.objects.all()
    loc_html = "".join([f'<p><i class="material-icons left">location_on</i> {l.city}, {l.address}</p>' for l in locations])

    if request.user.is_authenticated:
        auth = f'''<li><a href="/profile/" class="btn green darken-1 waves-effect waves-light" style="margin-right:10px;">📅 МОЇ ЗАПИСИ</a></li><li><a href="/logout/" class="red-text">ВИХІД ({request.user.username})</a></li>'''
        cta = '<a href="/book/" class="btn-large orange darken-3 pulse">ЗАПИСАТИСЯ</a>'
    else:
        auth = '<li><a href="/login/">Вхід</a></li><li><a href="/register/" class="orange-text">Реєстрація</a></li>'
        cta = '<a href="/register/" class="btn-large green darken-2">Стати клієнтом</a>'

    s_html = "".join([f'<div class="col s12 m4"><a href="/book/" style="color:inherit;"><div class="card hoverable z-depth-2"><div class="card-content center"><span class="card-title"><b>{s.name}</b></span><p class="grey-text">{s.description}</p><h5 class="green-text">{s.price} грн</h5><small class="orange-text">Натисніть для запису</small></div></div></a></div>' for s in services])
    m_html = "".join([f'<div class="col s12 m4"><a href="/master/{m.id}/" style="color:inherit;"><div class="card hoverable"><div class="card-image"><img src="{m.photo_url}" style="height:250px;object-fit:cover"></div><div class="card-content center"><span class="card-title"><b>{m.name}</b></span><p class="orange-text">{"★"*int(m.rating)} ({m.rating})</p><p class="grey-text">{m.reviews_count} відгуків</p><button class="btn-small grey darken-3" style="width:100%">Профіль</button></div></div></a></div>' for m in masters])
    map_html = '<iframe width="100%" height="300" frameborder="0" scrolling="no" src="https://www.openstreetmap.org/export/embed.html?bbox=30.518,50.448,30.525,50.453&amp;layer=mapnik&amp;marker=50.450,30.522"></iframe>'

    return HttpResponse(f'''<!DOCTYPE html><html lang="uk"><head><title>BarberShop</title><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"><style>body{{background:#f5f5f5;font-family:'Roboto',sans-serif}} .hero{{background:linear-gradient(rgba(0,0,0,0.7),rgba(0,0,0,0.7)),url('https://images.unsplash.com/photo-1585747860715-2ba37e788b70?ixlib=rb-1.2.1&auto=format&fit=crop&w=1950&q=80');background-size:cover;height:550px;display:flex;align-items:center;justify-content:center;color:white;text-align:center}} nav{{background:transparent;box-shadow:none;position:absolute;width:100%;z-index:999}} .section-header{{text-align:center;margin:60px 0 40px;text-transform:uppercase;letter-spacing:2px;font-weight:bold;color:#333}}</style></head><body>
        <nav><div class="nav-wrapper container"><a href="/" class="brand-logo"><i class="material-icons left">content_cut</i>BARBER</a><ul class="right hide-on-med-and-down"><li><a href="#services">Послуги</a></li><li><a href="#masters">Майстри</a></li><li><a href="#about">Про нас</a></li>{auth}</ul></div></nav>
        <div class="hero"><div class="container"><h1 style="font-weight:900;text-transform:uppercase">Твій Стиль.</h1><br>{cta}</div></div>
        <div id="services" class="container"><h4 class="section-header">Прайс-лист</h4><div class="center" style="margin-bottom:20px"><a href="?sort=asc" class="btn waves-effect grey darken-3">Дешевше</a> <a href="?sort=desc" class="btn waves-effect grey darken-3">Дорожче</a></div><div class="row">{s_html}</div></div>
        <div id="masters" class="grey lighten-3" style="padding:40px 0"><div class="container"><h4 class="section-header">Команда</h4><div class="row">{m_html}</div></div></div>
        <div id="about" class="container" style="padding:60px 0;text-align:center"><h4 class="section-header">Про нас</h4><p class="flow-text">Атмосфера, віскі, стиль.</p></div>
        <div class="grey darken-4 white-text" style="padding:60px 0"><div class="container"><div class="row"><div class="col s12 m6"><h5>Контакти</h5>{loc_html}<p><i class="material-icons left">phone</i> +380 99 123 45 67</p></div><div class="col s12 m6">{map_html}</div></div></div></div>
        <footer class="black page-footer"><div class="footer-copyright"><div class="container">© 2026 BarberShop</div></div></footer></body></html>''')

@csrf_exempt
def master_detail(request, master_id):
    master = get_object_or_404(Master, id=master_id)
    if request.method == "POST" and request.user.is_authenticated:
        Review.objects.create(master=master, user=request.user, rating=int(request.POST.get('rating')), text=request.POST.get('text'))
        avg = master.reviews.aggregate(Avg('rating'))['rating__avg']; master.rating = round(avg, 1) if avg else 5.0; master.reviews_count = master.reviews.count(); master.save()
        return redirect(f'/master/{master_id}/')
    revs = "".join([f'<li class="collection-item avatar"><i class="material-icons circle green">person</i><span class="title"><b>{r.user.username}</b></span><p class="orange-text">{"★"*r.rating}</p><p>{r.text}</p></li>' for r in master.reviews.all().order_by('-created_at')])
    form = f'''<div class="card-panel"><form method="POST"><label>Ваша оцінка:</label><select class="browser-default" name="rating" style="margin-bottom:15px"><option value="5">⭐⭐⭐⭐⭐ (5)</option><option value="4">⭐⭐⭐⭐ (4)</option><option value="3">⭐⭐⭐ (3)</option><option value="2">⭐⭐ (2)</option><option value="1">⭐ (1)</option></select><input name="text" required placeholder="Ваш відгук"><br><br><button class="btn green">Надіслати</button></form></div>''' if request.user.is_authenticated else '<div class="card-panel orange lighten-4 center">Увійдіть, щоб залишити відгук.</div>'
    return HttpResponse(f'''<!DOCTYPE html><html><head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"></head><body class="grey lighten-5">
    <nav class="grey darken-4"><div class="container"><a href="/" class="brand-logo left"><i class="material-icons">arrow_back</i> Назад</a></div></nav><div class="container" style="margin-top:40px"><div class="row"><div class="col s12 m4"><img src="{master.photo_url}" class="responsive-img z-depth-3" style="border-radius:10px"></div><div class="col s12 m8"><h3>{master.name}</h3><h5>{master.specialty}</h5><h4 class="orange-text">★ {master.rating} <small class="grey-text">({master.reviews_count} відгуків)</small></h4><p class="flow-text">{master.bio}</p><br><a href="/book/" class="btn-large orange darken-3">Записатися до майстра</a></div></div><div class="row"><div class="col s12"><h4>Відгуки клієнтів</h4>{form}<ul class="collection">{revs}</ul></div></div></div></body></html>''')

@login_required
def profile_view(request):
    bookings = Booking.objects.filter(user=request.user).order_by('-date_time')
    rows = "".join([f'<tr><td>{b.date_time.strftime("%d.%m.%Y %H:%M")}</td><td>{b.service.name}</td><td>{b.master.name}</td><td>{b.location.city}</td><td><span class="new badge green" data-badge-caption="Активно"></span></td></tr>' for b in bookings])
    if not rows: rows = "<tr><td colspan='5' class='center'>У вас ще немає записів.</td></tr>"
    return HttpResponse(f'''<!DOCTYPE html><html><head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"></head><body class="grey lighten-4"><nav class="grey darken-4"><div class="container"><a href="/" class="brand-logo left"><i class="material-icons">arrow_back</i> На головну</a></div></nav><div class="container" style="margin-top:40px"><div class="card-panel"><h4><i class="material-icons left">account_circle</i> Кабінет користувача</h4><p><b>Логін:</b> {request.user.username}</p><p><b>З нами з:</b> {request.user.date_joined.strftime("%d.%m.%Y")}</p></div><div class="card"><div class="card-content"><span class="card-title">📅 Історія записів</span><table class="striped highlight responsive-table"><thead><tr><th>Дата</th><th>Послуга</th><th>Майстер</th><th>Місто</th><th>Статус</th></tr></thead><tbody>{rows}</tbody></table></div></div></div></body></html>''')

@csrf_exempt
@login_required
def book_service(request):
    if request.method == "POST":
        Booking.objects.create(user=request.user, service_id=request.POST.get('s'), master_id=request.POST.get('m'), location_id=request.POST.get('l'), date_time=request.POST.get('d'), phone=request.POST.get('phone'), wishes=request.POST.get('w'))
        return redirect('/profile/')

    l_ops = "".join([f'<option value="{x.id}">{x.city}, {x.address}</option>' for x in Location.objects.all()])
    s_ops = "".join([f'<option value="{x.id}">{x.name} - {x.price} грн</option>' for x in Service.objects.all()])
    m_ops = "".join([f'<option value="{x.id}">{x.name}</option>' for x in Master.objects.all()])

    return HttpResponse(f'''<!DOCTYPE html><html><head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"><style>select{{display:block;margin-bottom:10px}}</style></head><body class="grey lighten-4"><div class="container" style="margin-top:20px"><div class="row"><div class="col s12 m8 offset-m2"><div class="card"><div class="card-content"><a href="/" class="btn-flat waves-effect" style="margin-bottom:10px; padding-left:0"><i class="material-icons left">arrow_back</i>Скасувати</a><h4>Запис</h4><form method="POST"><label>Послуга</label><select name="s">{s_ops}</select><label>Майстер</label><select name="m">{m_ops}</select><label>Локація</label><select name="l">{l_ops}</select><label>Телефон</label><input name="phone" required placeholder="+380..."><label>Час</label><input type="datetime-local" name="d" required><label>Побажання</label><textarea name="w" class="materialize-textarea"></textarea><br><br><button class="btn-large green waves-effect waves-light" style="width:100%">ПІДТВЕРДИТИ</button></form></div></div></div></div></div></body></html>''')

@csrf_exempt
def register_view(request):
    if request.method == "POST":
        form = UserCreationForm(request.POST)
        if form.is_valid(): user = form.save(); login(request, user); return redirect('/profile/')
    else: form = UserCreationForm()
    return HttpResponse(f'''<html><head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"></head><body class="valign-wrapper" style="height:100vh; background:#eee"><div class="container"><div class="row"><div class="col s12 m6 offset-m3"><div class="card z-depth-4" style="padding:20px"><a href="/" class="btn-flat waves-effect" style="margin-bottom:10px; padding-left:0"><i class="material-icons left">arrow_back</i>На головну</a><h4 class="center">Реєстрація</h4><form method="POST">{form.as_p()}<br><button class="btn-large green waves-effect waves-light" style="width:100%">Створити акаунт</button></form><div class="clearfix"></div></div></div></div></div></body></html>''')

@csrf_exempt
def login_view(request):
    if request.method == "POST":
        form = AuthenticationForm(data=request.POST)
        if form.is_valid(): user = form.get_user(); login(request, user); return redirect('/profile/')
    else: form = AuthenticationForm()
    return HttpResponse(f'''<html><head><link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css"><link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"></head><body class="valign-wrapper" style="height:100vh; background:#eee"><div class="container"><div class="row"><div class="col s12 m6 offset-m3"><div class="card z-depth-4" style="padding:20px"><a href="/" class="btn-flat waves-effect" style="margin-bottom:10px; padding-left:0"><i class="material-icons left">arrow_back</i>На головну</a><h4 class="center">Вхід</h4><form method="POST">{form.as_p()}<br><button class="btn-large orange waves-effect waves-light" style="width:100%">Увійти</button></form><div class="clearfix"></div><br><center><a href="/register/">Немає акаунту?</a></center></div></div></div></div></body></html>''')

def logout_view(request): logout(request); return redirect('/')
"""
with open(os.path.join(inner_path, 'views.py'), 'w') as f: f.write(views_code)

# 10. URLs
urls_code = """
from django.contrib import admin
from django.urls import path
from .views import home, register_view, login_view, logout_view, book_service, master_detail, profile_view
urlpatterns = [path('admin/', admin.site.urls), path('', home), path('register/', register_view), path('login/', login_view), path('logout/', logout_view), path('book/', book_service), path('master/<int:master_id>/', master_detail), path('profile/', profile_view)]
"""
with open(os.path.join(inner_path, 'urls.py'), 'w') as f: f.write(urls_code)

# 11. Run
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'barbershop_project.settings')
django.setup()

local_db = 'db.sqlite3'
if drive_path and os.path.exists(drive_path):
    shutil.copy(drive_path, local_db)
    !python manage.py migrate
else:
    !python manage.py makemigrations barbershop_project
    !python manage.py migrate

    pop = """
import os, django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'barbershop_project.settings')
django.setup()
from barbershop_project.models import Location, Master, Service, Review
from django.contrib.auth.models import User

User.objects.filter(username__iexact='taras').delete()
if not User.objects.filter(username='admin').exists(): User.objects.create_superuser('admin', 'admin@example.com', 'admin123')

if Location.objects.count() == 0:
    l1=Location.objects.create(city="Київ", address="вул. Хрещатик, 1")
    l2=Location.objects.create(city="Львів", address="Площа Ринок, 10")
    m1=Master.objects.create(name="Олексій", specialty="Top Barber", location=l1, rating=5.0, reviews_count=10, photo_url="https://images.unsplash.com/photo-1595152772835-219674b2a8a6?w=500")
    m2=Master.objects.create(name="Андрій", specialty="Beard King", location=l1, rating=4.8, reviews_count=5, photo_url="https://images.unsplash.com/photo-1605497788044-5a32c7078486?w=500")
    m3=Master.objects.create(name="Макс", specialty="Junior", location=l2, rating=4.2, reviews_count=2, photo_url="https://images.unsplash.com/photo-1583337130417-3346a1be7dee?w=500")
    Service.objects.create(name="Стрижка", price=500, description="Класика"); Service.objects.create(name="Борода", price=300)
"""
    with open('pop.py', 'w') as f: f.write(pop)
    !python pop.py
    if drive_path: shutil.copy(local_db, drive_path)

def autosave():
    while True:
        time.sleep(15)
        try:
            if drive_path: shutil.copy(local_db, drive_path)
        except: pass
threading.Thread(target=autosave, daemon=True).start()

print(f"Site: {public_url}")
print(f"Admin: {public_url}/admin")
!python manage.py runserver 8000

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  No migrations to apply.
Site: https://shoeless-unreferenced-emile.ngrok-free.dev
Admin: https://shoeless-unreferenced-emile.ngrok-free.dev/admin
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
January 06, 2026 - 21:17:21
Django version 6.0.1, using settings 'barbershop_project.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.

For more information on production servers see: https://docs.djangoproject.com/en/6.0/howto/deployment/
Session data corrupted
[06/Jan/2026 21:17:22] "GET / HTTP/1.1" 200 6159
[06/Jan/2026 21:17:28] "GET /login/ HTTP/1.1" 200 1251
Session data corrupted
[06/Jan/2026 21:17:38] "POST /login/ HTTP/1.1" 302 0
[06/Jan/202